In [7]:
# Ensure project root is on sys.path so `config.py` can be imported when running from the notebooks folder
import sys, os
nb_dir = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'notebooks' else os.getcwd()
proj_root = nb_dir if os.path.isfile(os.path.join(nb_dir, 'config.py')) else os.path.dirname(nb_dir)
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)
print('Using project root:', proj_root)

Using project root: c:\Users\arnav\Desktop\Arnav\ROG\Arnav\Arnav\python\ExoPatternNet


In [8]:
# ...existing code...
import os, pandas as pd, numpy as np
import config
import matplotlib.pyplot as plt
import lightkurve as lk
from config import DEFAULTS

# ...existing code...
# Resolve processed data directories
search_dirs = [
    os.path.join(proj_root, "data", "processed"),
    os.path.join(proj_root, "notebooks", "data", "processed"),
]
existing_dirs = [d for d in search_dirs if os.path.isdir(d)]
print("Search dirs:", existing_dirs)

# Find flattened CSVs across dirs
candidates = []
for d in existing_dirs:
    candidates += [os.path.join(d, f) for f in os.listdir(d) if f.endswith("_flattened.csv")]

assert candidates, f"No flattened CSVs in any of: {existing_dirs}"
path = sorted(candidates)[-1]
DATA_PRO = os.path.dirname(path)
print("Using CSV:", path)

# Read CSV and construct a LightCurve
df = pd.read_csv(path)
need = {"time", "flux"}
missing = need - set(df.columns)
assert not missing, f"CSV missing columns: {missing}. Found: {df.columns.tolist()}"
lc = lk.LightCurve(
    time=df["time"].to_numpy(),
    flux=df["flux"].to_numpy(),
    flux_err=df["flux_err"].to_numpy() if "flux_err" in df.columns else None,
    quality=df["quality"].to_numpy() if "quality" in df.columns else None,
)

# BLS transit search
periodogram = lc.to_periodogram(
    method="bls",
    period=np.linspace(DEFAULTS["min_period_days"], DEFAULTS["max_period_days"], 5000),
    duration=DEFAULTS["duration_hours_candidates"],
)
ax = periodogram.plot()
ax.set_title("Box Least Squares Periodogram")
plt.show()

print("Best period (days):", periodogram.period_at_max_power)
print("Best duration (days):", periodogram.duration_at_max_power)
# ...existing code...

Search dirs: ['c:\\Users\\arnav\\Desktop\\Arnav\\ROG\\Arnav\\Arnav\\python\\ExoPatternNet\\data\\processed', 'c:\\Users\\arnav\\Desktop\\Arnav\\ROG\\Arnav\\Arnav\\python\\ExoPatternNet\\notebooks\\data\\processed']
Using CSV: c:\Users\arnav\Desktop\Arnav\ROG\Arnav\Arnav\python\ExoPatternNet\notebooks\data\processed\Kepler-10_Kepler_flattened.csv


AssertionError: CSV missing columns: {'time'}. Found: ['flux', 'flux_err', 'quality']